In [1]:
from utils.utils import set_seed
import torch
set_seed(42)

Seed fixada em 42


c:\Users\User\Desktop\Maths\Projects\HodgeAndSheaf\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Dataset

In [2]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root = "/tmp/MUTAG", name = "MUTAG")

Dataset properties

In [3]:
print(dataset)
print(f"number of graphs: {len(dataset)}")
print(f"number of classes: {dataset.num_classes}")
print(f"number of node features: {dataset.num_node_features}")
print(f"number of edge features: {dataset.num_edge_features}")

MUTAG(188)
number of graphs: 188
number of classes: 2
number of node features: 7
number of edge features: 4


dataset shapes

In [4]:
print(dataset._data)

Data(x=[3371, 7], edge_index=[2, 7442], edge_attr=[7442, 4], y=[188])


In [5]:
from sklearn.model_selection import StratifiedKFold
from torch_geometric.loader import DataLoader
import numpy as np
def create_data_splits(dataset, n_splits=10, batch_size=32, random_seed=42):
    """
    Cria splits para validação cruzada k-fold estratificada.
    
    A estratificação garante que cada fold tenha aproximadamente a mesma
    proporção de classes que o dataset completo. Isso é importante porque
    temos um desbalanceamento de classes no MUTAG.
    """
    # Extraindo os labels para estratificação
    y = np.array([data.y.item() for data in dataset])
    
    # Criando o objeto de validação cruzada
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_seed)
    
    # Lista para armazenar os loaders de cada fold
    fold_loaders = []
    
    for fold_idx, (train_val_idx, test_idx) in enumerate(skf.split(np.zeros(len(y)), y)):
        # Dividimos train_val_idx em treino e validação (80/20)
        # Usamos estratificação novamente aqui
        inner_skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_seed)
        train_idx, val_idx = next(inner_skf.split(
            np.zeros(len(train_val_idx)), 
            y[train_val_idx]
        ))
        
        # Convertendo índices relativos para absolutos
        train_idx = train_val_idx[train_idx]
        val_idx = train_val_idx[val_idx]
        
        # Criando subsets
        train_dataset = dataset[train_idx.tolist()]
        val_dataset = dataset[val_idx.tolist()]
        test_dataset = dataset[test_idx.tolist()]
        
        # Criando DataLoaders
        # O DataLoader do PyG automaticamente faz o batching de múltiplos grafos
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        
        fold_loaders.append({
            'train': train_loader,
            'val': val_loader,
            'test': test_loader,
            'train_size': len(train_dataset),
            'val_size': len(val_dataset),
            'test_size': len(test_dataset)
        })
        
        if fold_idx == 0:
            print(f'Fold {fold_idx + 1}:')
            print(f'  Treino: {len(train_dataset)} grafos')
            print(f'  Validação: {len(val_dataset)} grafos')
            print(f'  Teste: {len(test_dataset)} grafos')
    
    return fold_loaders

# Criando os splits
fold_loaders = create_data_splits(dataset, n_splits=10, batch_size=32)

Fold 1:
  Treino: 135 grafos
  Validação: 34 grafos
  Teste: 19 grafos


In [6]:
class GraphClassificationTrainer:
    def __init__(self, model, fold_loaders, config, device='cpu'):
        self.model = model.to(device)
        self.fold_loaders = fold_loaders
        self.config = config
        self.device = device
        
        # Configurando otimizador
        if config['optimizer'] == 'adam':
            self.optimizer = torch.optim.Adam(
                model.parameters(),
                lr=config['lr'],
                weight_decay=config['weight_decay']
            )
        
        # Scheduler de learning rate
        if config.get('scheduler') == 'step':
            self.scheduler = torch.optim.lr_scheduler.StepLR(
                self.optimizer,
                step_size=config['step_size'],
                gamma=config['gamma']
            )
        elif config.get('scheduler') == 'cosine':
            self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
                self.optimizer,
                T_max=config['epochs']
            )
        else:
            self.scheduler = None
        
        self.criterion = torch.nn.CrossEntropyLoss()
        
        # Para early stopping
        self.best_val_acc = 0
        self.patience_counter = 0
        self.best_model_state = None
    
    def train_epoch(self, train_loader):
        """
        Treina por uma época completa, iterando sobre todos os batches.
        """
        self.model.train()
        total_loss = 0
        correct = 0
        total = 0
        
        for batch in train_loader:
            # Move o batch para o device apropriado (CPU ou GPU)
            batch = batch.to(self.device)
            
            self.optimizer.zero_grad()
            
            # Forward pass
            # Note que agora passamos batch.batch para indicar os limites dos grafos
            out = self.model(batch.x, batch.edge_index, batch.batch)
            
            # Calcula a perda
            loss = self.criterion(out, batch.y)
            
            # Backward pass
            loss.backward()
            
            # Gradient clipping se configurado
            if self.config.get('clip_grad'):
                torch.nn.utils.clip_grad_norm_(
                    self.model.parameters(),
                    self.config['clip_grad']
                )
            
            self.optimizer.step()
            
            # Acumula estatísticas
            total_loss += loss.item() * batch.num_graphs
            pred = out.argmax(dim=1)
            correct += (pred == batch.y).sum().item()
            total += batch.num_graphs
        
        avg_loss = total_loss / total
        accuracy = correct / total
        
        return avg_loss, accuracy
    
    @torch.no_grad()
    def evaluate(self, loader):
        """
        Avalia o modelo em um DataLoader específico.
        """
        self.model.eval()
        correct = 0
        total = 0
        
        for batch in loader:
            batch = batch.to(self.device)
            out = self.model(batch.x, batch.edge_index, batch.batch)
            pred = out.argmax(dim=1)
            correct += (pred == batch.y).sum().item()
            total += batch.num_graphs
        
        return correct / total
    
    def train_fold(self, fold_idx):
        """
        Treina o modelo em um fold específico da validação cruzada.
        """
        print(f'\n=== Treinando Fold {fold_idx + 1} ===')
        
        # Reseta o modelo para cada fold
        self.model.apply(self._weight_reset)
        
        # Reseta otimizador e scheduler
        self.__init__(self.model, self.fold_loaders, self.config, self.device)
        
        loaders = self.fold_loaders[fold_idx]
        train_loader = loaders['train']
        val_loader = loaders['val']
        test_loader = loaders['test']
        
        history = {
            'train_loss': [],
            'train_acc': [],
            'val_acc': [],
            'test_acc': []
        }
        
        self.best_val_acc = 0
        self.patience_counter = 0
        
        for epoch in range(1, self.config['epochs'] + 1):
            # Treina por uma época
            train_loss, train_acc = self.train_epoch(train_loader)
            
            # Avalia em validação e teste
            val_acc = self.evaluate(val_loader)
            test_acc = self.evaluate(test_loader)
            
            # Guarda histórico
            history['train_loss'].append(train_loss)
            history['train_acc'].append(train_acc)
            history['val_acc'].append(val_acc)
            history['test_acc'].append(test_acc)
            
            # Atualiza learning rate
            if self.scheduler is not None:
                self.scheduler.step()
            
            # Early stopping
            if self.config.get('early_stopping'):
                if val_acc > self.best_val_acc:
                    self.best_val_acc = val_acc
                    self.best_model_state = self.model.state_dict().copy()
                    self.patience_counter = 0
                else:
                    self.patience_counter += 1
                
                if self.patience_counter >= self.config['patience']:
                    print(f'Early stopping na época {epoch}')
                    self.model.load_state_dict(self.best_model_state)
                    break
            
            # Log periódico
            if epoch % self.config['log_interval'] == 0:
                print(f'Época {epoch:03d}: Loss={train_loss:.4f}, '
                      f'Train={train_acc:.4f}, Val={val_acc:.4f}, Test={test_acc:.4f}')
        
        # Avaliação final no melhor modelo
        if self.best_model_state is not None:
            self.model.load_state_dict(self.best_model_state)
        
        final_val_acc = self.evaluate(val_loader)
        final_test_acc = self.evaluate(test_loader)
        
        print(f'Melhor Val Acc: {final_val_acc:.4f}')
        print(f'Test Acc final: {final_test_acc:.4f}')
        
        return history, final_val_acc, final_test_acc
    
    def train_all_folds(self):
        """
        Executa validação cruzada completa em todos os folds.
        """
        val_accs = []
        test_accs = []
        all_histories = []
        
        for fold_idx in range(len(self.fold_loaders)):
            history, val_acc, test_acc = self.train_fold(fold_idx)
            val_accs.append(val_acc)
            test_accs.append(test_acc)
            all_histories.append(history)
        
        # Calcula estatísticas agregadas
        mean_val_acc = np.mean(val_accs)
        std_val_acc = np.std(val_accs)
        mean_test_acc = np.mean(test_accs)
        std_test_acc = np.std(test_accs)
        
        print('\n' + '='*50)
        print('RESULTADOS DA VALIDAÇÃO CRUZADA')
        print('='*50)
        print(f'Validação: {mean_val_acc:.4f} ± {std_val_acc:.4f}')
        print(f'Teste: {mean_test_acc:.4f} ± {std_test_acc:.4f}')
        print('='*50)
        
        return {
            'val_accs': val_accs,
            'test_accs': test_accs,
            'mean_val_acc': mean_val_acc,
            'std_val_acc': std_val_acc,
            'mean_test_acc': mean_test_acc,
            'std_test_acc': std_test_acc,
            'histories': all_histories
        }
    
    @staticmethod
    def _weight_reset(m):
        """
        Reseta os pesos de uma camada para reinicialização.
        """
        if hasattr(m, 'reset_parameters'):
            m.reset_parameters()

In [7]:
# Precisamos importar nossa classe ConfigurableGNN
# (assumindo que você salvou o código anterior)
import torch
from models.GraphNeuralNetwork.GraphConvolutionalNetwork import GCN
# Detecta se temos GPU disponível
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Usando device: {device}')

# Configuração do modelo
# Note que precisamos usar pooling='mean' ou 'max' para classificação de grafos
model = GCN(
    num_features=dataset.num_node_features,
    num_classes=dataset.num_classes,
    hidden_dims=[64, 64, 32],  # 3 camadas ocultas
    conv_type='GCN',
    activation='relu',
    dropout=0.5,
    batch_norm=True,
    residual=False,  # Não usamos residual para rede de apenas 3 camadas
    pooling='mean',  # CRUCIAL: pooling para agregar nós em representação do grafo
    jk_mode=None
)

# Configuração do treinamento
config = {
    'optimizer': 'adam',
    'lr': 0.001,  # Learning rate um pouco menor para dataset pequeno
    'weight_decay': 1e-4,
    'scheduler': 'step',
    'step_size': 50,
    'gamma': 0.5,
    'epochs': 200,
    'clip_grad': 1.0,
    'early_stopping': True,
    'patience': 25,  # Paciência maior porque o dataset é pequeno
    'log_interval': 20
}

# Criando o trainer
trainer = GraphClassificationTrainer(model, fold_loaders, config, device=device)

# Executando validação cruzada completa
results = trainer.train_all_folds()

Usando device: cpu

=== Treinando Fold 1 ===
Época 020: Loss=0.5501, Train=0.7333, Val=0.8235, Test=0.8947
Época 040: Loss=0.5158, Train=0.7185, Val=0.8235, Test=0.9474
Early stopping na época 42
Melhor Val Acc: 0.8235
Test Acc final: 0.9474

=== Treinando Fold 2 ===
Época 020: Loss=0.5167, Train=0.7333, Val=0.6471, Test=0.7895
Early stopping na época 39
Melhor Val Acc: 0.6765
Test Acc final: 0.7368

=== Treinando Fold 3 ===
Época 020: Loss=0.5103, Train=0.7556, Val=0.7647, Test=0.6842
Época 040: Loss=0.4722, Train=0.7778, Val=0.8235, Test=0.6316
Época 060: Loss=0.4642, Train=0.7704, Val=0.7941, Test=0.6316
Early stopping na época 62
Melhor Val Acc: 0.8235
Test Acc final: 0.6316

=== Treinando Fold 4 ===
Época 020: Loss=0.5146, Train=0.7630, Val=0.6765, Test=0.7895
Early stopping na época 26
Melhor Val Acc: 0.6765
Test Acc final: 0.8421

=== Treinando Fold 5 ===
Época 020: Loss=0.4804, Train=0.7407, Val=0.7941, Test=0.6316
Early stopping na época 38
Melhor Val Acc: 0.7941
Test Acc fina